In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense
import tensorflow.compat.v1 as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import Dense, Conv2D, Flatten, Dropout,BatchNormalization,MaxPooling2D


In [4]:
import csv

data = {}
with open('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    
    # first row in csv has column headers, we dont want column headers. 
    next(reader)
    
    # now getting our key-value pair
    for row in reader:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [5]:
data.keys()

dict_keys(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [6]:
distraction_list = list(data.keys())

In [7]:
# so we need to make master_data directory.
# inside it, we create two subdirectories training and testing.

import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')
os.mkdir('master_data/validation')


In [8]:
# inside the training and testing, we need to make 10 new subdirectories related to these distractions.
# so we iterate over each distractions.
for distraction in distraction_list:
    os.mkdir(os.path.join('master_data/training/', distraction))
    os.mkdir(os.path.join('master_data/testing/', distraction))
    os.mkdir(os.path.join('master_data/validation/', distraction))

# we have created subdirectiories inside training and testing but the directories are empty so we need to fill them
# with source directory images.

In [9]:
import shutil as sh
split_size = 0.7
val_size  = 0.1

for dis, images in data.items():
    train_size = int(split_size*len(images))
    val_size = int(val_size*len(images))
    train_images = images[:train_size]
    val_images = images[train_size:val_size]
    test_images = images[train_size + val_size:]
    for image in train_images:
        source = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train',  dis,image)
        dest = os.path.join('master_data/training',   dis)
        sh.copy(source, dest)
    for image in test_images:
        source = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train', dis, image)
        dest = os.path.join('master_data/testing',  dis)
        sh.copy(source, dest)
        
    for image in val_images:
        source = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train', dis, image)
        dest = os.path.join('master_data/validation',  dis)
        sh.copy(source, dest)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'master_data/training'
test_dir = 'master_data/testing'
val_dir = 'master_data/validation'


train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                train_dir,   # directory from which we want to generate our data.
                                target_size = (100, 100),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 128
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                test_dir,   # directory from which we want to generate our data.
                                target_size = (100, 100),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 128
)

val_datagen = ImageDataGenerator(rescale = 1.0/255)
val_generator = val_datagen.flow_from_directory(
                                val_dir,   # directory from which we want to generate our data.
                                target_size = (100, 100),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 128
)

Found 15692 images belonging to 10 classes.
Found 499 images belonging to 10 classes.
Found 5985 images belonging to 10 classes.


In [11]:
es = EarlyStopping(monitor = 'loss', patience = 2, min_delta=0.01)


In [19]:
model = tf.keras.models.Sequential([
    # layer 1
    Conv2D(16, (3,3), activation = 'relu', input_shape = (64, 64, 1)),
    BatchNormalization(),
    Conv2D(32,(3,3), activation = 'relu', padding = 'same'), 
    BatchNormalization(axis = 3),
    MaxPooling2D(pool_size = (2,2), padding = 'same'),
    Dropout(0.3),
    
    # layer 2
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'), 
    BatchNormalization(axis = 3),
    MaxPooling2D(pool_size = (2,2), padding = 'same'),
    Dropout(0.3),
    
    #layer 3
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'), 
    BatchNormalization(axis = 3),
    MaxPooling2D(pool_size = (2,2), padding = 'same'),
    Dropout(0.5),
    
    # output
    Flatten(),
    Dense(512, activation ='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation = 'relu'),
    Dropout(0.25),
    
    # Last Layer is Important coz we have to decide here mow many classes are there in the images
    Dense(10, activation ='softmax')


    
])

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 16)        160       
_________________________________________________________________
batch_normalization_7 (Batch (None, 62, 62, 16)        64        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 62, 62, 32)        4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 62, 62, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 31, 31, 64)       

In [22]:
model.fit(train_generator, epochs = 10, verbose = 1, validation_data = val_generator , callbacks = [es])

Epoch 1/10


NotFoundError:  No algorithm worked!
	 [[node sequential_1/conv2d_6/Conv2D (defined at <ipython-input-21-174bcdc9750c>:1) ]] [Op:__inference_train_function_5745]

Function call stack:
train_function
